<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
case class RGB(channelWidth: Int) extends Bundle{
  val red   = UInt(channelWidth bit)
  val green = UInt(channelWidth bit)
  val blue  = UInt(channelWidth bit)
  def isBlack : Bool = red === 0 && green === 0 && blue === 0
} 

class MyRGB extends Component{
  val source = slave  Stream(RGB(8))
  val sink   = master Stream(RGB(8))
  sink <-< source.s2mPipe
} 

showRtl(new MyRGB)class T3 extends Component{
 val a = in(UInt(8 bits))
 val b = out(UInt(8 bits))
 b := a + 133
}

In [ ]:
class Myflow1 extends Component{
    val source = slave  Flow(RGB(8))
    val sink   = master Flow(RGB(8))
    sink <-< source 
}

showRtl(new Myflow1)

In [ ]:
class T1 extends Component{
    val a = slave Flow(UInt(8 bits) )
    val b = master Flow(UInt())
    val tmp = a.toStream
    b := tmp.toFlow
}
showRtl(new T1)

In [ ]:
class T1 extends Component{
    val a = slave Flow(UInt(8 bits) )
    val b = master Stream(UInt())
    definitionName ="myT1"
    b << a.toStream
}
showRtl(new T1)

In [ ]:
class T1 extends Component{
    val a = slave Stream(UInt(8 bits))
    val b = master Flow(UInt(8 bits) ) 
    b << a.toFlow
}
showRtl(new T1)

In [ ]:
case class FilterConfig(iqWidth: Int, 
                        tapNumbers: Int = 33,
                        hwFreq: HertzNumber = 200 MHz, 
                        sampleFreq: HertzNumber = 1.92 MHz)

case class IQ(width: Int) extends Bundle{
  val I,Q = SInt(width bits)
}

class Filter(fc: FilterConfig) extends Component{
  //val din   = slave Flow(IQ(fc.iqWidth))
  //val dout  = master Flow(IQ(fc.iqWidth))
  val din   = slave Flow(Bits(32 bits))
  val dout  = master Flow(Bits(32 bits))
  val flush = in Bool
    
  val clockSMP = ClockDomain.external("smp")
  val clockHW = ClockDomain.external("hw")
    
  val u_fifo_in = StreamFifoCC(
      dataType = Bits(32 bits), 
      depth = 8,
      pushClock = clockSMP,
      popClock = clockDomain
  )
    
  u_fifo_in.io.push << din.toStream 
  dout << u_fifo_in.io.pop.toFlow
    
}

showRtl(new Filter(FilterConfig(8)))

增加OUT

In [ ]:
class IQ extends Bundle{
  val I, Q = SInt(8 bits)
}
class Filter extends Component{
    val din = slave Flow(new IQ)    
    val dout = master Stream(new IQ)
    
    val u_asyncfifo = StreamFifoCC(
    dataType = din.payload,
    depth = 8,
    pushClock = ClockDomain.external("smp"),
    popClock = clockDomain)
    
    u_asyncfifo.io.push << din.toStream
    dout << u_asyncfifo.io.pop
}
showRtl(new Filter)

In [ ]:
class IQ extends Bundle{
  val I, Q = SInt(8 bits)
}
class Filter extends Component{
    val din = slave Flow(new IQ)    
    val dout = master Stream(new IQ)
    dout << din.toStream.queue(16)
}
showRtl(new Filter)

In [ ]:
class IQin extends Bundle 
class T2 extends Component{
    val din = slave Flow(new IQ)
    val din0 = in(new IQ)
}
showRtl(new T2)

In [ ]:
case class Color(channelWidth: Int) extends Bundle {
  val r,g,b = UInt(channelWidth bits)
}
class T3 extends Component{
  val io = new Bundle{
   val input  = in (Color(8) )
   val output = out(Color(8))
}
    io.output <> io.input
}
showRtl(new T3)

In [ ]:
class T3 extends Component{
    val a = in UInt(8 bits)
    val b = in SInt(8 bits)
    val c = out SInt()
    c := a.resize(a.getWidth + 1).asSInt + b
}

showRtl(new T3)

In [ ]:
class T3 extends Component{
    val a = in UInt(8 bits)
    val b = in SInt(8 bits)
    val c = out SInt()
    c := S(a.resize(a.getWidth + 1)) + b
}

showRtl(new T3)

In [ ]:
println(((132.5 MHz).toTime/(1E-9)).toInt)

In [ ]:
println(9/7.0)

## rgb

In [ ]:
 
case class Rgb(channelWidth: Int) extends Bundle{
  val r = UInt(channelWidth bits)
  val g = UInt(channelWidth bits)
  val b = UInt(channelWidth bits)
  def init(x: Int): Rgb = {
      val ret = cloneOf(this)
      ret.r := U(x)
      ret.g := U(x)
      ret.b := U(x)
      ret 
  }
  def clear = {
      this.r := 0
      this.g := 0
      this.b := 0
  }
 // override def clone :Rgb = Rgb.asInstanceOf[this.type]
}

class T3 extends Component{
    val a = slave Flow(Rgb(8))     
    val flush = in Bool()
    val b = master Flow(Rgb(8))
    
    val retReg = RegNext(a.payload) init 0   //how to 
    
    when(flush){
       retReg.clear
    } 
    
    b.payload := retReg
    b.valid := True
 
}
showRtl(new T3)

## StreamFIFO
- 深度大于1的 Mem输出readSync readyAsync可选
- occupanccy/avalibility 可选

In [ ]:
showRtl(StreamFifo(UInt(8 bits),1))

In [ ]:
class Top extends Component{
 //val a = slave  Stream(UInt( 8 bits)) 
 //val b = master Stream(UInt(8 bits))
 //    b := a.queue(4)
    Stream(UInt(8 bits)).queue(4)
}
showRtl(new Top)

## Mem 
- readSync  下拍输出
- readAsync 当拍输出

In [ ]:
class Top extends Component{
    val addr = in UInt(5 bits)
  val ram = Mem(UInt(8 bits),32)
    val b = out UInt(8 bits)
    b := ram.readAsync(addr) 
    //b := ram.readSync(addr) 
}
showRtl(new Top)

In [ ]:
class Top extends Component{
    val a = slave Flow(Fragment(UInt(8 bits)))
}
showRtl(new Top)